In [1]:
import os
import copy
import wget
import time
import asyncio
import warnings
import logging

import numpy as np

from lsst.ts import salobj
from lsst.ts.standardscripts.auxtel.attcs import ATTCS

%matplotlib inline

In [2]:
attcs = ATTCS()
cam = salobj.Remote(attcs.domain, "GenericCamera", 1)

await asyncio.gather(attcs.start_task, cam.start_task) 

RemoteEvent(ATDomeTrajectory, 0, logMessage) falling behind; read 100 messages
RemoteTelemetry(ATDome, 0, position) falling behind; read 27 messages


[[None, None, None, None, None, None, None], None]

RemoteTelemetry(ATPtg, 0, timeAndDate) falling behind; read 13 messages
RemoteTelemetry(ATPtg, 0, mountStatus) falling behind; read 14 messages
RemoteTelemetry(ATPtg, 0, guidingAndOffsets) falling behind; read 14 messages
RemoteTelemetry(ATPtg, 0, currentTargetStatus) falling behind; read 16 messages
RemoteEvent(ATMCS, 0, target) falling behind; read 35 messages


In [3]:
# Declare target name
target_name='HD32309'

In [4]:
await attcs.slew_icrs(ra = "5 01 25.58", dec = "-20 03 06.9", rot_pa=0., target_name=target_name)

RemoteEvent(ATMCS, 0, target) falling behind; read 52 messages
RemoteEvent(ATDomeTrajectory, 0, logMessage) falling behind; read 53 messages
RemoteTelemetry(ATDome, 0, position) falling behind; read 14 messages
RemoteTelemetry(ATPtg, 0, currentTargetStatus) falling behind; read 13 messages
RemoteTelemetry(ATPtg, 0, timeAndDate) falling behind; read 13 messages
RemoteTelemetry(ATPtg, 0, mountStatus) falling behind; read 14 messages
RemoteTelemetry(ATPtg, 0, guidingAndOffsets) falling behind; read 14 messages


TimeoutError: 

In [13]:
rotation_matrix = lambda angle: np.array([
            [np.cos(np.radians(angle)), -np.sin(np.radians(angle)), 0.],
            [np.sin(np.radians(angle)), np.cos(np.radians(angle)), 0.],
            [0., 0., 1.]])
azel = await attcs.atmcs.tel_mount_AzEl_Encoders.aget()
nasmyth = await attcs.atmcs.tel_mount_Nasmyth_Encoders.aget()
angle = np.mean(azel.elevationCalculatedAngle)+np.mean(nasmyth.nasmyth1CalculatedAngle)
el, az, _ = np.matmul([20, -18., 0.], rotation_matrix(angle))

In [14]:
el, az

(22.639904254724527, 14.540795553782687)

In [15]:
await attcs.atptg.cmd_offsetAzEl.set_start(az=az, el=el, num=0)

In [16]:
5*360/24

75.0

In [ ]:
attcs.slew_icrs?

In [ ]:
await attcs.slew_icrs

In [ ]:
await attcs.ataos.cmd_applyAxisOffset.set_start(axis="x", offset=x_offset)
await attcs.ataos.cmd_applyAxisOffset.set_start(axis="y", offset=y_offset)

In [ ]:
await attcs.ataos.cmd_applyAxisOffset.set_start(axis="z", offset=focus_offset)

In [ ]:
dz = 0.8

try:
    await cam.cmd_stopLiveView.start()
except salobj.AckError:
    print("ack error")

# Intrafocal hexapod setting - offsets -0.5mm from central focus value above

print('Move to intrafocal position')
# await attcs.athexapod.cmd_moveToPosition.set_start(x=hex_X, y=hex_Y,
#                                              z=hex_Z-dz, u=hex_U, v=hex_V)
attcs.athexapod.evt_positionUpdate.flush()
await attcs.ataos.cmd_applyAxisOffset.set_start(axis="z", offset=-dz+focus_offset)
curr_hex_pos = await attcs.athexapod.evt_positionUpdate.next(flush=False, timeout=30)

print('Take intrafocal image')
cam.evt_endReadout.flush()

await cam.cmd_takeImages.set_start(numImages=1, expTime=expTime, shutter=True, imageSequenceName=target_name+'_intra')

intra_endron = await cam.evt_endReadout.next(flush=False, timeout=5)

# Extrafocal hexapod setting
print('Move to extra-focal hexapod position')
# await attcs.athexapod.cmd_moveToPosition.set_start(x=hex_X, y=hex_Y,
#                                                      z=hex_Z+dz, u=hex_U, v=hex_V)
attcs.athexapod.evt_positionUpdate.flush()
await attcs.ataos.cmd_applyAxisOffset.set_start(axis="z", offset=dz+focus_offset)
curr_hex_pos = await attcs.athexapod.evt_positionUpdate.next(flush=False, timeout=30)

# r.evt_endReadout.flush()
print('Take extra-focal image')
cam.evt_endReadout.flush()
await cam.cmd_takeImages.set_start(numImages=1, expTime=expTime, shutter=True, imageSequenceName=target_name+'_extra')

extra_endron = await cam.evt_endReadout.next(flush=False, timeout=5)

azel = await attcs.atmcs.tel_mount_AzEl_Encoders.aget()
nasmyth = await attcs.atmcs.tel_mount_Nasmyth_Encoders.aget()

print(f"============")
print(f"dz={dz}")
print(intra_endron.imageName)
print(extra_endron.imageName)
print(f"============")

try:
    await cam.cmd_startLiveView.set_start(expTime=0.5)
except salobj.AckError:
    print("ack error")

attcs.athexapod.evt_positionUpdate.flush()
await attcs.ataos.cmd_applyAxisOffset.set_start(axis="z", offset=focus_offset)
curr_hex_pos = await attcs.athexapod.evt_positionUpdate.next(flush=False, timeout=30)

print(f"['{intra_endron.imageName}.fits'], ['{extra_endron.imageName}.fits']")
print(np.mean(azel.elevationCalculatedAngle)+np.mean(nasmyth.nasmyth1CalculatedAngle))

# intra_wget_url = 'http://139.229.170.216:8000/data/'+intra_endron.imageName+'.fits'
# extra_wget_url = 'http://139.229.170.216:8000/data/'+extra_endron.imageName+'.fits'
# intra_fname = wget.download(intra_wget_url)
# print(f"============")
# print(f'{intra_fname}')
# extra_fname = wget.download(extra_wget_url)
# print(f'{extra_fname}')
# print(f"============")


In [ ]:
intra_wget_url = 'http://139.229.170.216:8000/data/'+intra_endron.imageName+'.fits'
extra_wget_url = 'http://139.229.170.216:8000/data/'+extra_endron.imageName+'.fits'

intra_fname = wget.download(intra_wget_url, out="/home/saluser/develop/cwfs/python/data")
extra_fname = wget.download(extra_wget_url, out="/home/saluser/develop/cwfs/python/data")

print(intra_fname,extra_fname)

In [ ]:
await attcs.ataos.cmd_offset.set_start(z=0.)

In [ ]:
await attcs.ataos.cmd_resetOffset.start()

In [ ]:
await attcs.ataos.cmd_offset.set_start(z=-0.3)